<span style="font-size:8pt">*ÉTS Montréal, GPA671 : Introduction à l’intelligence artificielle. Date : le 23/10/21. Auteur : Jérôme Rony. Version : 1.2*</span>

# Laboratoire 2 : Classificateurs statistiques et SVM

## Exercice 1 : SVM linéaire à marge souple

Dans cet exercice, le but est d’implémenter un SVM linéaire à marge souple. Afin de pouvoir visualiser les frontières de décision, les données seront dans un espace à 2 dimensions.

### Rappels de cours

Dans un SVM à marge souple, on cherche à maximiser la marge tout en minimisant les variables ressorts $\xi_i$, ce qui correspond au problème suivant :
\begin{equation}
\begin{aligned}
\underset{{\bf w}, b, \xi}{\text{minimize}}&& \frac{1}{2} {\bf w}^\top {\bf w} &+ C\sum\limits_{n=1}^{N}\xi_n\\
\text{subject to}&& \quad y_n({\bf w}^\top {\bf x}_n + b) & \geq 1 - \zeta_i  &n = 1, \dots, N\\
&& \xi_n & \geq 0 &n=1, \dots, N
\end{aligned}
\end{equation}

Avec C qui est un hyper-paramètre qui contrôle le compromis entre les deux objectifs de l’entrainement. Ce problème d’optimisation peut être reformulé en un problème équivalent :
\begin{equation}
\underset{{\bf w}, b}{\text{minimize}} \quad \frac{1}{2} {\bf w}^\top {\bf w} + C\sum\limits_{n=1}^{N}\max(0, 1 - y_n({\bf w}^\top {\bf x}_n + b))
\end{equation}

### Algorithme PEGASOS

Résoudre l’un des deux problèmes précédents n’est pas simple dans le cas général, et requiert des méthodes et des heuristiques d’optimisation avancées. Une méthode récente pour résoudre un équivalent du deuxième problème est l’algorithme PEGASOS. Encore une fois, on reformule le problème précédent en un autre problème équivalent :
\begin{equation}
\underset{{\bf w}, b}{\text{minimize}} \quad \frac{\lambda}{2} {\bf w}^\top {\bf w} + \frac{1}{N}\sum\limits_{n=1}^{N}\max(0, 1 - y_n({\bf w}^\top {\bf x}_n + b))
\end{equation}

Cette formulation est proche de la précédente, mais l'hyper-paramètre $C$ a été remplacé par $\lambda$, qui contrôle le compromis opposé. De plus, la somme est maintenant divisée par le nombre d'exemples. Ces modifications mineures permettent d'obtenir des notations plus simples dans la suite.

Dans l’algorithme PEGASOS, on initialise le vecteur de poids ${\bf w}$ aléatoirement, tel que $||{\bf w}|| \leq 1 / \sqrt{\lambda}$. Ensuite, à chaque itération de l’algorithme, on sélectionne aléatoirement un sous ensemble de $k$ exemples d’entrainement et on effectue une itération de descente de gradient du problème précédent. Enfin on contraint la norme du nouveau vecteur de poids $\tilde{{\bf w}}$ avec : ${\bf w}^{(t+1)} = \min\left\{1, \dfrac{1}{\sqrt{\lambda}||\tilde{{\bf w}}||}\right\}\tilde{{\bf w}}$.

L’algorithme est le suivant :
* **Entrées** : 
    * Données ${\bf x}_n$ avec étiquettes $y_n\in\{-1, +1\}$ pour $n=1, \dots, N$
    * Hyper-paramètre de compromis $\lambda$.
    * Nombre d’itérations $T$
    * Nombre d’exemples choisis aléatoirement dans chaque itération : $k$
* **Initialisation** : 
    * Vecteur de poids ${\bf w}$ initialisé aléatoirement tel que $||{\bf w}|| \leq \frac{1}{\sqrt{\lambda}}$
    * Biais initialisé à zéro : $b = 0$
* Pour $t = 1, \dots, T$:
    1. Choisir aléatoirement $k$ exemples parmi les $N$ données d’entrainement. On appelle $A_t$ l’ensemble de ces $k$ exemples, donc $|A_t|=k$.
    2. Trouver le sous-ensemble $A_t^+ = \{({\bf x}, y)\in A_t : y({\bf w}^\top {\bf x} + b) < 1\}$
    3. Calculer les gradients de $f_t({\bf w}, b, A_t) = \frac{\lambda}{2}{\bf w}^\top {\bf w} + \frac{1}{k}\sum\limits_{({\bf x}, y)\in A_t}\max\{0, 1 - y({\bf w}^\top {\bf x} + b)\}$ par rapport à ${\bf w}$ et $b$ :
    \begin{equation}
    \begin{aligned}
        \nabla_{\bf w} f_t &= \lambda {\bf w} - \frac{1}{k} \sum\limits_{({\bf x}, y)\in A_t^+} y {\bf x}\\
        \nabla_b f_t &= - \frac{1}{k} \sum\limits_{({\bf x}, y)\in A_t^+} y
    \end{aligned}
    \end{equation}
    4. Effectuer un ajustement de ${\bf w}$ et $b$ par descente de gradient avec un taux d’apprentissage $\eta = \dfrac{1}{\lambda t}$ : 
    \begin{equation}
    \tilde{{\bf w}} = {\bf w} - \eta \nabla_{\bf w} f_t  \qquad b = b - \eta \nabla_b f_t
    \end{equation}
    5. Ajuster la norme de ${\bf w}$ telle que ${\bf w}^{(t+1)} = \min\left\{1, \dfrac{1}{\sqrt{\lambda}||\tilde{{\bf w}}||}\right\}\tilde{{\bf w}}$.
* **Sorties** : vecteur de poids ${\bf w}$ et biais $b$.

### Questions

1. Dans un premier temps, nous allons générer des données synthétiques que l’on peut presque séparer linéairement, c’est-à-dire avec du chevauchement, à l’aide du code suivant :
```python
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X, y = make_classification(n_samples=1000, n_features=2, n_informative=1, n_redundant=0, n_clusters_per_class=1, random_state=42, shift=1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.8, random_state=42)
```
Afficher ces données à l’aide de la fonction `scatter` de Matplotlib avec une couleur différente pour chaque classe.
2. Implémenter l’algorithme PEGASOS présenté précédemment dans la classe `LinearSVM` fournie qui a les méthodes suivantes :
    * `fit` qui prend des données d’entrainement (`X` et `y`) et applique l’algorithme PEGASOS pour apprendre ${\bf w}$ et $b$.
    * `decision_function` qui prend en entrée des données `X` et renvoie le score ${\bf w}^\top {\bf x} + b$ pour chaque donnée.  
    * `predict` qui prend en entrée des données `X` et renvoie l'étiquette prédite correspondant au signe de ${\bf w}^\top {\bf x} + b$.
> Attention : les labels générés par `make_classification` sont des entiers positifs alors qu'un SVM utilise les labels $-1$ et $+1$.
3. À l’aide des exemples de validation pour obtenir le taux d’exactitude, trouver les meilleurs hyper-paramètres en essayant les valeurs $\lambda \in \{0.0001, 0.001, 0.01, 0.1, 1, 10, 100\}$ et $k\in\{1, 2, 5, 10, 20, 50, 100, 200\}$, avec $T = 1000$. Afficher un graphique résumant les performances obtenues.
4. En s’inspirant de [la documentation de scikit-learn](https://scikit-learn.org/stable/auto_examples/svm/plot_linearsvc_support_vectors.html#sphx-glr-auto-examples-svm-plot-linearsvc-support-vectors-py), afficher la frontière de décision, les marges et vecteurs de supports par-dessus les données d’entrainement.
5. Générer des nouvelles données à l’aide de du code suivant :
```python
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=1000, noise=0.2, random_state=42)
X, X_val, y, y_val = train_test_split(X, y, test_size=0.8, random_state=42)
```
Entrainer le SVM linéaire sur ces données et calculer le taux d’exactitude sur les données de validation. Afficher les données, la frontière de décision, les marges et les vecteurs de support dans un graphique. Commenter le taux d’exactitude.

In [ ]:
import numpy as np

class LinearSVM(object):
    def __init__(self, lambd: float, k: int, T: int) -> None:
        """
        Classificateur SVM entrainé avec l'algorithme PEGASOS.
        
        Parameters
        ----------
        lambd : float
            Paramètre de compromis entre la maximisation de la marge et le respect des contraintes du SVM.
        k : int
            Nombre d'exemples aléatoirement choisis parmis les données d'entrainement dans chaque itération.
        T : int
            Nombre d'itérations d'entrainement.

        """
        pass

    def fit(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        """
        Entrainement du SVM.
        
        Parameters
        ----------
        X_train : np.ndarray
            Données d'entrainement. Taille : (# exemples, # entrées).
        y_train : np.ndarray
            Étiquettes des données d'entrainement. Taille : (# exemples,).

        """
        pass
    
    def decision_function(self, X: np.ndarray) -> np.ndarray:
        """
        Évaluation de la fonction de décision du SVM linéaire : w^T x + B.

        Parameters
        ----------
        X_train : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
            
        Returns
        -------
        scores : np.ndarray
            Score(s) pour chaque donnée d'entrée. Taille : (# exemples,).
        
        """
        pass

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du SVM.
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        y_pred : np.ndarray
            Étiquette(s) prédite(s) pour chaque donnée d'entrée. Taille : (# exemples,).

        """
        pass

## Exercice 2 : SVM avec noyau et marge souple

Dans cet exercice, le but est d’implémenter un SVM avec un noyau Gaussien (aussi appelé *Radial Basis Function* ou *RBF*) et marge souple.

### Rappels de cours

Dans un SVM à noyau, les vecteurs ${\bf x}_n$ sont projetés dans un nouvel espace par la fonction $\phi$ pour obtenir ${\bf z}_n = \phi({\bf x}_n)$. Cependant, ce nouveau vecteur peut possiblement avoir une dimension infinie (ce qui est le cas pour un noyau Gaussien). On ne veut / peut donc pas calculer explicitement ces vecteurs. Pour maximiser la marge avec les contraintes comme précédemment, on ne peut donc plus résoudre le problème primal ; on doit passe donc au *dual*. On résout alors le problème suivant :
\begin{equation}
\begin{aligned}
\underset{\alpha}{\text{maximize}}&& \sum\limits_{n=1}^{N} \alpha_n &- \frac{1}{2}\sum\limits_{n=1}^{N}\sum\limits_{m=1}^{N}\alpha_n\alpha_m y_n y_m K({\bf x}_n, {\bf x}_m)\\
\text{subject to}&& 0 \leq \alpha_n &\leq C \quad n = 1, \dots, N\\
&& \sum\limits_{n=1}^N \alpha_n y_n &= 0
\end{aligned}
\end{equation}

Ici, $\alpha\in\mathbb{R}_+^N$ est un vecteur qui donne un "poids" à chaque donnée d’entrainement, $K$ est la fonction noyau et C est l’hyper-paramètre qui contrôle le compromis entre les deux objectifs de l’entrainement. Sous certaines conditions, on peut montrer que $K({\bf x}_n, {\bf x}_m) = {\bf z}_n^\top {\bf z}_m = \phi({\bf x}_n)^\top \phi({\bf x}_m)$. C’est-à-dire que $K$ permet d’exprimer la ressemblance entre deux vecteurs dans un espace transformé (possiblement à dimension infinie), sans jamais explicitement transformer les vecteurs d’entrées ${\bf x}_n$. 

Dans cet exercice, on va utiliser le noyau Gaussien, qui est le plus courant, définit par :
\begin{equation}
K({\bf x}_n, {\bf x}_m) = \exp(-\gamma ||{\bf x}_n - {\bf x}_m||^2)
\end{equation}
Ici, $\gamma$ correspond à l’inverse de deux fois la déviation standard au carré $\gamma = \frac{1}{2\sigma^2}$. Son choix est critique pour la performance ; une bonne approximation (utilisée par défaut dans scikit-learn) est : $\gamma = \dfrac{1}{\text{# entrées} \times {\mathrm{Var}(X)}}$.

Une fois le problème ci-dessus résolu, on obtient un vecteur $\alpha\in\mathbb{R}_+^N$. Chaque composante de ce vecteur est positive ou nulle et les données ${\bf x}_n$ pour lesquelles $\alpha_n > 0$ sont appelés des **vecteurs de support**. On peut alors effectuer une prediction pour un nouveau vecteur ${\bf x}$:
\begin{equation}
y_{\text{pred}} = \sum\limits_{n=1}^{N}\alpha_n y_n K({\bf x}, {\bf x}_n) + b
\end{equation}
Où $b$ est le biais. On peut remarquer que le biais n’apparait pas dans le problème résolu précédemment. Pour le trouver, on utilise l’équation suivante :
\begin{equation}
b = \frac{1}{|\mathcal{S}|}\sum\limits_{n\in\mathcal{S}}\left(y_n - \sum\limits_{m=1}^N \alpha_m y_m K({\bf x}_n, {\bf x}_m)\right)
\end{equation}
où $\mathcal{S} = \{n : \alpha_n \neq 0, n=1,\dots,N\}$ est l’ensemble des indices des vecteurs de support.

### Algorithme PEGASOS

Résoudre exactement le problème dual ci-dessus requiert, encore une fois, des méthodes et des heuristiques d’optimisation avancées. L’algorithme PEGASOS est une méthode simple pour résoudre ce problème de manière approximative. Cet algorithme ne permet pas d’obtenir la meilleure solution possible, mais à l’avantage d’être simple à implémenter, léger en calcul et fournit des résultats assez bons.

L’algorithme est le suivant :
* **Entrées** : 
    * Données ${\bf x}_n$ avec étiquettes $y_n\in\{-1, +1\}$ pour $n=1, \dots, N$
    * Hyper-paramètre de compromis $\lambda$.
    * Nombre d’itérations $T$
    * Nombre d’exemples choisis aléatoirement dans chaque itération : $k$
* **Initialisation** : 
    * Vecteur $\beta^{(0)}$ initialisé à ${\bf 0}$.
* Pour $t = 1, \dots, T$:
    1. Choisir aléatoirement $k$ exemples parmi les $N$ données d’entrainement. On appelle $A_t$ l’ensemble des **indices** de ces $k$ exemples, donc $|A_t|=k$.
    2. **Pour** $n = 1, \dots, N$:  
        * **Si** &nbsp; $n\in A_t$ &nbsp; **et** &nbsp; $\dfrac{y_n}{\lambda t} \sum\limits_{m=1}^{N} \beta_m y_m K({\bf x}_m, {\bf x}_n) < 1$ &nbsp; **alors** &nbsp; $\beta_n^{(t)} = \beta_n^{(t-1)} + 1$  
        * **Sinon** &nbsp; $\beta_n^{(t)} = \beta_n^{(t-1)}$  
* Calcul des $\alpha$ finaux : $\alpha_n = \frac{1}{\lambda T}\beta_n^{(T)}$
* Ensemble des indices des vecteurs de support : $\mathcal{S} = \{n : \alpha_n \neq 0, n=1,\dots,N\}$
* Calcul du biais : $b = \dfrac{1}{|\mathcal{S}|}\sum\limits_{n\in\mathcal{S}}\left(y_n - \sum\limits_{m=1}^N \alpha_m y_m K({\bf x}_n, {\bf x}_m)\right)$
* **Sorties** : vecteur $\alpha$, ensemble des vecteurs de support $\mathcal{S}$ et biais $b$.

### Questions

Dans cet exercice, nous allons réutiliser les données obtenues lors de la question 5 de l’exercice 3.

1. Implémenter l’algorithme PEGASOS présenté précédemment dans la classe `KernelSVM` fournie. Cette classe a les méthodes suivantes :
    * `fit` qui prend des données d’entrainement (`X` et `y`) et applique l’algorithme PEGASOS pour apprendre les $\alpha_n$ et le biais $b$.
    * `decision_function` qui prend en entrée des données `X` et renvoie le score $\sum_{n\in\mathcal{S}}\alpha_n y_n K({\bf x}, {\bf x}_n) + b$ pour chaque donnée.  
    * `predict` qui prend en entrée des données `X` et renvoie l'étiquette prédite correspondant au signe de $\sum_{n\in\mathcal{S}}\alpha_n y_n K({\bf x}, {\bf x}_n) + b$.
> Attention : les labels générés par `make_classification` sont des entiers positifs alors qu’un SVM utilise les labels $-1$ et $+1$.
2. À l’aide des exemples de validation pour obtenir le taux d’exactitude, trouver les meilleurs hyper-paramètres en essayant les valeurs $\lambda \in \{0.0001, 0.001, 0.01, 0.1, 1, 10, 100\}$ et $k\in\{1, 2, 5, 10, 20, 50, 100, 200\}$, avec $T = 1000$. Afficher un graphique résumant les performances obtenues.
3. Afficher la frontière de décision, les marges et vecteurs de supports par-dessus les données d’entrainement.
4. Comparer le résultat obtenu avec le SVM de scikit-learn `sklearn.svm.SVC` quantitativement et qualitativement. Commenter la position des vecteurs de support par rapport à la marge dans les deux cas et le choix de $C$ pour `sklearn.svm.SVC` par rapport à $\lambda$.

In [ ]:
import numpy as np

class KernelSVM(object):
    def __init__(self, lambd: float, k: int, T: int) -> None:
        """
        Classificateur SVM avec noyau entrainé avec l'algorithme PEGASOS.
        
        Parameters
        ----------
        lambd : float
            Paramètre de compromis entre la maximisation de la marge et le respect des contraintes du SVM.
        k : int
            Nombre d'exemples aléatoirement choisis parmis les données d'entrainement dans chaque itération.
        T : int
            Nombre d'itérations d'entrainement.

        """
        pass

    def fit(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        """
        Entrainement du SVM.
        
        Parameters
        ----------
        X_train : np.ndarray
            Données d'entrainement. Taille : (# exemples, # entrées).
        y_train : np.ndarray
            Étiquettes des données d'entrainement. Taille : (# exemples,).

        """
        pass
    
    def decision_function(self, X: np.ndarray) -> np.ndarray:
        """
        Évaluation de la fonction de décision du SVM avec noyau.

        Parameters
        ----------
        X_train : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
            
        Returns
        -------
        scores : np.ndarray
            Score(s) pour chaque donnée d'entrée. Taille : (# exemples,).
        
        """
        pass

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du SVM.
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        y_pred : np.ndarray
            Étiquette(s) prédite(s) pour chaque donnée d'entrée. Taille : (# exemples,).

        """
        pass

## Exercice 3 : kNN

Pour cet exercice, nous allons implémenter un classificateur des k plus proches voisins (*k-Nearest Neighbors* ou *k-NN*).

### Rappels de cours

Le classificateur des k plus proches voisins est un algorithme qui classifie les objets selon leur proximité aux données d’entraînement. C’est une approche de modélisation simple qui approxime la frontière de décision localement. Il n’y a pas d’entraînement à proprement parler et les calculs sont seulement effectués lors de la classification. Comme la figure ci-dessous le montre, la classification d’un exemple est effectuée en choisissant la classe la plus fréquente parmi les plus proches voisins. Bien que plusieurs mesures de distance puissent être utilisées pour définir le voisinage, la mesure la plus utilisée est la distance Euclidienne, définie par :
\begin{equation*}
d(p,q) = \sqrt{\sum_{i=1}^{d}(p_i - q_i)^2}
\end{equation*}

où $p$ et $q$ sont deux points dans un espace ${\mathbb R}^d$.

<img src=images/kNN.png style="width: 300px;">

Dans cette figure, si on ne considère que 3 voisins on attribuera la classe A alors que si on considère 7 voisins, on attribuera plutôt la classe B.

Normalement, un kNN ne génère pas de probabilités, car il renvoie simplement la classe majoritaire. Dans cet exercice, on va modifier la fonction de classification pour introduire la notion de probabilité.
\begin{equation*}
P(C_j | {\bf x}) = \dfrac{n_j}{k}
\end{equation*}
où $n$ est le nombre de plus proches voisins appartenant à la classe $C_j$ et $k$ le nombre total de plus proches voisins considérés.

Le classificateur kNN est une méthode non paramétrique qui ne nécessite pas d’établir une hypothèse au préalable sur la nature des distributions de données (contrairement à un Perceptron par exemple). Le seul paramètre à déterminer est la taille du voisinage $k$. On choisit ce paramètre à partir des données.

### Questions

1. Charger les données du datatset Iris à l’aide de la fonction `sklearn.datasets.load_iris`.
2. Séparer aléatoirement les données en deux parties (entrainement et test) de manière balancée avec la fonction `sklearn.model_selection.train_test_split`.
3. Implémenter un kNN permettant d’utiliser n’importe quel $k$ en paramètre en utilisant la classe fournie ci-dessous :
    * La méthode `fit` sert à charger les données et déterminer les classes du problème.
    * La méthode `predict_proba` prend en entrée des données `X` et renvoie les probabilités prédites pour chaque classe du problème.
    * La méthode `predict` prend en entrée des données `X` et renvoie la classe prédite pour chaque échantillon.
4. Calculer et tracer le taux d'exactitude pour 1-NN, 2-NN jusqu’à 10-NN. Commenter ces résultats. Quels sont les limitations de cette méthode d’évaluation ?

In [ ]:
import numpy as np

class kNN(object):
    def __init__(self, k: int) -> None:
        """
        Classificateur kNN.
        
        Parameters
        ----------
        k : int
            Nombre de plus proches voisins utilisés pour classifier un nouveau point.

        """
        pass

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Chargement des données.
        
        Parameters
        ----------
        X : np.ndarray
            Données d'entrainement. Taille : (# exemples, # entrées).
        y : np.ndarray
            Étiquettes des données d'entrainement. Taille : (# exemples,).

        """
        pass

    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du kNN pour un ou plusieurs points.
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        probabilities : np.ndarray
            Vecteur(s) de probabilités prédits pour chaque données d'entrée. Taille : (# exemples, # classes).

        """
        pass

    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Fonction de classification du kNN pour un ou plusieurs points
        
        Parameters
        ----------
        X : np.ndarray
            Donnée(s) d'entrée à classifier. Taille : (# exemples, # entrées).
        
        Returns
        -------
        y_pred : np.ndarray
            Étiquette(s) prédite(s) pour chaque donnée d'entrée.. Taille : (# exemples,).

        """
        pass

## Exercice 4 (Bonus) : Classificateur Bayésien naïf

Le but de cet exercice est d’implémenter un classificateur Bayésien naïf dans lequel on fait les suppositions suivantes :
 * Les caractéristiques du problème sont indépendantes ;
 * Les caractéristiques suivent des distributions Gaussiennes :
     \begin{equation}
     P(x | \mu_k, \sigma_k) = \dfrac{1}{\sigma_k \sqrt{2\pi}}\exp\left(-\dfrac{(x - \mu_k)^2}{2\sigma_k^2}\right)
     \end{equation}
   où $\mu_k$ et $\sigma_k$ sont la moyenne et l’écart-type d’une composante pour la classe $k$.

### Questions

Dans cet exercice, on réutilise les données de l’exercice précédent (dataset Iris) avec la même séparation.

1. Implémenter un classificateur Bayésien naïf dans une classe `NaiveBayes`. Cette classe contient les mêmes méthodes que la classe `kNN` donnée ci-dessus. Ici la méthode `fit` permet de déterminer les classes du problème, calculer les probabilités a priori et calculer les caractéristiques de chaque classe (pour pouvoir calculer les probabilités conditionnelles dans `predict_proba`).
2. Calculer le taux d’exactitude pour ce classificateur et le comparer au meilleur des kNN obtenu précédemment.  Commenter ce taux et conclure quant aux différences de modélisations entre ces deux modèles. Quels sont les avantages du classificateur Bayésien naïf par rapport au kNN ?